# Model 2: Part 2

## PanelOLS $AR_{i,t}=\alpha + \beta_1 Log_case_{i,t}+ \beta_2 Return_{i,t} + \beta_3 ReturnM_{i,t} + FE + \epsilon_{i,t}$

In [7]:
import warnings
warnings.filterwarnings("ignore")

import matplotlib
matplotlib.rcParams['axes.labelsize'] = 14
matplotlib.rcParams['xtick.labelsize'] = 12
matplotlib.rcParams['ytick.labelsize'] = 12
matplotlib.rcParams['text.color'] = 'k'

from itertools import product
from datetime import datetime
import matplotlib.pyplot as plt
plt.style.use('classic')
plt.rcParams['font.sans-serif']=['Microsoft YaHei'] 
import numpy as np
import pandas as pd
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import statsmodels.api as sm
from linearmodels.panel import PanelOLS
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, median_absolute_error, mean_squared_log_error

In [8]:
df = pd.read_excel(r'E:\Jupyter_Notebook\Modeling Competition_Master\Stock Market_ESM_PanelOLS\China_PanelOLS_Data.xlsx')
data = df.set_index(['indices','day'])
data.head()

dummy trading_day  month     AR  log_case    return   returnM  case  cure  death
indices             day                                                                                  
china_tansportation 0        1  2020-01-21      1 -0.149  5.003946 -1.826378 -1.723387   149     0      3
                    1        1  2020-01-22      1 -0.420  4.875197 -0.051570  0.427347   131     0      8
                    2        1  2020-01-23      1 -0.140  5.556828 -3.171558 -3.147572   259     6      8
                    3        1  2020-02-03      2 -1.765  8.081475 -9.609919 -8.208697  3234   155     64
                    4        1  2020-02-04      2 -3.181  8.265393 -0.743353  2.603305  3887   262     65

In [9]:
VIFs = pd.DataFrame()
VIFs["VIF Factor"] = [variance_inflation_factor(data[['return','returnM','case']].values, i)
                     for i in range(data[['return','returnM','case']].shape[1])] # 'cure','death'
VIFs

,VIF Factor
0,4.121161
1,4.120539
2,1.023148


In [10]:
data[['AR','case','cure','death']].corr(method='pearson')['AR']

AR       1.000000
case    -0.180411
cure     0.075779
death   -0.216932
Name: AR, dtype: float64

In [11]:
exog_vars = ['log_case','return','returnM']
exog = sm.add_constant(data[exog_vars])
mod = PanelOLS(data.AR, exog, entity_effects=True)
FE_res = mod.fit(cov_type="clustered", cluster_entity=True)
FE_res

Dep. Variable:,AR,R-squared:,0.9977
Estimator:,PanelOLS,R-squared (Between):,0.9985
No. Observations:,70,R-squared (Within):,0.9977
Date:,"Fri, Aug 14 2020",R-squared (Overall):,0.9977
Time:,13:59:38,Log-likelihood,109.39
Cov. Estimator:,Clustered,,
,,F-statistic:,9515.1
Entities:,2,P-value,0.0000
Avg Obs:,35.000,Distribution:,"F(3,65)"
Min Obs:,35.000,,
Max Obs:,35.000,F-statistic (robust):,-2.121e+18


### Estimation Results:$https://bashtage.github.io/linearmodels/doc/panel/results.html#linearmodels.panel.results.PanelEffectsResults$